<a href="https://colab.research.google.com/github/Prateekbangwal/machine_learning/blob/master/dogs_vs_cat_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2020-06-23 16:01:43--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 2404:6800:4008:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   160MB/s    in 0.4s    

2020-06-23 16:01:44 (160 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape =(150,150,3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation = 'relu'),
                                    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [ ]:
model.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr = 0.0001),
              metrics = ['accuracy'])

In [ ]:
# All images will be rescaled by 1./255
# train_datagen = ImageDataGenerator(rescale = 1/255)
train_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = False,
    fill_mode = 'nearest'
)
test_datagen = ImageDataGenerator(rescale = 1/255)

In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary'
)
# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 100,# 2000 images = batch_size * steps
    epochs = 100,
    validation_data = validation_generator,
    validation_steps = 50, # 1000 images = batch_size * steps
    verbose = 2

)

Epoch 1/100
100/100 - 111s - loss: 0.6086 - accuracy: 0.7010 - val_loss: 0.5282 - val_accuracy: 0.7500
Epoch 2/100
100/100 - 115s - loss: 0.5545 - accuracy: 0.7115 - val_loss: 0.4910 - val_accuracy: 0.7600
Epoch 3/100
100/100 - 110s - loss: 0.5407 - accuracy: 0.7175 - val_loss: 0.4790 - val_accuracy: 0.7650
Epoch 4/100
100/100 - 110s - loss: 0.5323 - accuracy: 0.7310 - val_loss: 0.4969 - val_accuracy: 0.7630
Epoch 5/100
100/100 - 110s - loss: 0.5266 - accuracy: 0.7275 - val_loss: 0.4992 - val_accuracy: 0.7600
Epoch 6/100
100/100 - 110s - loss: 0.5222 - accuracy: 0.7320 - val_loss: 0.4569 - val_accuracy: 0.7850
Epoch 7/100
100/100 - 115s - loss: 0.5078 - accuracy: 0.7515 - val_loss: 0.4610 - val_accuracy: 0.7700
Epoch 8/100
100/100 - 114s - loss: 0.5151 - accuracy: 0.7395 - val_loss: 0.4584 - val_accuracy: 0.7900
Epoch 9/100
100/100 - 113s - loss: 0.5185 - accuracy: 0.7420 - val_loss: 0.4748 - val_accuracy: 0.7590
Epoch 10/100
100/100 - 113s - loss: 0.5081 - accuracy: 0.7525 - val_loss:

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_acc']
loss =history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: ignored